# side model

In [1]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_3h.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(15059, 3000)
(15059, 7)


12h_dft_dom_cycle_lag26       301
12h_dft_dom_cycle_lag25       297
12h_dft_dom_cycle_lag24       293
12h_dft_dom_cycle_lag23       289
12h_dft_dom_cycle_dt_lag22    289
                             ... 
3h_comb_spectrum_pwr_35         0
3h_comb_spectrum_pwr_36         0
3h_forecast_oscillator          0
3h_roofing_filter               0
3h_voss                         0
Length: 3000, dtype: int64

In [2]:
df_features = df_features[301:]
label_side = label_side[301:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(14758, 3000)
(14758, 7)


45m_ac_33                    0
12h_adaptive_cci_ddt_lag2    0
12h_adaptive_bp_lead_lag6    0
12h_adaptive_bp_lead_lag7    0
12h_adaptive_bp_lead_lag8    0
                            ..
3h_adaptive_bp_dt_lag2       0
3h_adaptive_bp_dt_lag3       0
3h_adaptive_bp_dt_lag4       0
3h_adaptive_bp_lag1          0
12h_williams_r_lag4          0
Length: 3000, dtype: int64

In [3]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(14758, 3000)
(array([0, 1]), array([7083, 7675]))
(array([0, 1]), array([1402, 1550]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-25 18:42:26,015] A new study created in memory with name: no-name-a4672bef-0098-4918-ab15-e8566ebe77b7
[I 2025-03-25 18:42:32,185] Trial 0 finished with value: 0.8637264737011643 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 174, 'max_depth': 407, 'min_gain_to_split': 0.5352672482373497, 'min_data_in_leaf': 387, 'lambda_l1': 76.00887285419344, 'lambda_l2': 55.58124125529257, 'num_boost_round': 640}. Best is trial 0 with value: 0.8637264737011643.


precision: 0.7716436637390214, recall: 0.7935483870967742, f1: 0.7824427480916031


[I 2025-03-25 18:43:01,528] Trial 1 finished with value: 0.8632736643504673 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 278, 'max_depth': 264, 'min_gain_to_split': 0.1769062321694738, 'min_data_in_leaf': 32, 'lambda_l1': 37.24214155191833, 'lambda_l2': 74.06428297955429, 'num_boost_round': 1175}. Best is trial 0 with value: 0.8637264737011643.


precision: 0.7801047120418848, recall: 0.7690322580645161, f1: 0.774528914879792


[I 2025-03-25 18:43:06,385] Trial 2 finished with value: 0.8637669688463485 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 37, 'max_depth': 848, 'min_gain_to_split': 0.3629465358916214, 'min_data_in_leaf': 321, 'lambda_l1': 76.70719722392666, 'lambda_l2': 13.288476142070193, 'num_boost_round': 849}. Best is trial 2 with value: 0.8637669688463485.


...

precision: 0.776500638569604, recall: 0.7845161290322581, f1: 0.7804878048780488


[I 2025-03-25 18:53:10,654] Trial 98 finished with value: 0.8732616998757536 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 181, 'max_depth': 784, 'min_gain_to_split': 0.7480809993572761, 'min_data_in_leaf': 114, 'lambda_l1': 6.5628580864069, 'lambda_l2': 32.43144613244543, 'num_boost_round': 207}. Best is trial 11 with value: 0.8754714463209239.


precision: 0.7942332896461337, recall: 0.7819354838709678, f1: 0.788036410923277


[I 2025-03-25 18:53:13,868] Trial 99 finished with value: 0.8732317886889697 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 159, 'max_depth': 798, 'min_gain_to_split': 0.892132186914238, 'min_data_in_leaf': 130, 'lambda_l1': 13.381289534825822, 'lambda_l2': 28.465793116263463, 'num_boost_round': 216}. Best is trial 11 with value: 0.8754714463209239.


precision: 0.7920209287115761, recall: 0.7812903225806451, f1: 0.7866190321532965


In [6]:
study.best_params

{'is_unbalance': False,
 'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 220,
 'max_depth': 615,
 'min_gain_to_split': 0.6351384995613452,
 'min_data_in_leaf': 206,
 'lambda_l1': 0.9415247011279284,
 'lambda_l2': 35.77641135012539,
 'num_boost_round': 229}

In [7]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [8]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8754714463209239


In [10]:
model.save_model("model/model_side.txt")
prod_model.save_model("model/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_3h.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

12h_dft_dom_cycle_lag39    354
12h_dft_dom_cycle_lag38    350
12h_dft_dom_cycle_lag37    346
12h_dft_dom_cycle_lag36    342
12h_dft_dom_cycle_lag35    338
                          ... 
3h_ac_46                     0
3h_ac_37                     0
3h_ac_36                     0
3h_adaptive_rsi              0
side_model_res               0
Length: 3000, dtype: int64

In [2]:
df_features = df_features[354:]
meta_label = meta_label[354:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(14705, 3000)
(14705, 7)


45m_ac_10                     0
12h_adaptive_rsi_ddt_lag22    0
12h_adaptive_cci_lag34        0
12h_adaptive_cci_lag35        0
12h_adaptive_cci_lag36        0
                             ..
3h_dft_dom_cycle_dt_lag14     0
3h_dft_dom_cycle_dt_lag22     0
3h_dft_dom_cycle_dt_lag24     0
3h_dft_dom_cycle_dt_lag25     0
side_model_res                0
Length: 3000, dtype: int64

In [3]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(14705, 3000)
(array([0, 1]), array([ 1562, 10202]))
(array([0, 1]), array([ 631, 2310]))


In [4]:
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-25 19:09:06,645] A new study created in memory with name: no-name-c6601adf-70b1-44d1-8c4c-5c1448753ee0
[I 2025-03-25 19:09:37,781] Trial 0 finished with value: 0.7553319865694839 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 496, 'max_depth': 823, 'min_gain_to_split': 0.5203435461951008, 'min_data_in_leaf': 95, 'lambda_l1': 0.36224244322902316, 'lambda_l2': 33.69972571442159, 'num_boost_round': 1627}. Best is trial 0 with value: 0.7553319865694839.


precision: 0.8327828241123039, recall: 0.8731601731601731, f1: 0.8524936601859678, auc: 0.7521950315928129, fbeta: 0.7553319865694839


[I 2025-03-25 19:10:04,639] Trial 1 finished with value: 0.6542213693812868 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 109, 'max_depth': 609, 'min_gain_to_split': 0.5101072865331494, 'min_data_in_leaf': 33, 'lambda_l1': 12.672542885661864, 'lambda_l2': 86.12095383735299, 'num_boost_round': 1955}. Best is trial 0 with value: 0.7553319865694839.


precision: 0.9181879648411089, recall: 0.5878787878787879, f1: 0.7168118237001847, auc: 0.7516818627753652, fbeta: 0.6542213693812868


[I 2025-03-25 19:10:18,319] Trial 2 finished with value: 0.7315390801838917 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 87, 'max_depth': 239, 'min_gain_to_split': 0.3487520260142888, 'min_data_in_leaf': 29, 'lambda_l1': 27.851769871936725, 'lambda_l2': 77.03668090765967, 'num_boost_round': 1048}. Best is trial 0 with value: 0.7553319865694839.


...

precision: 0.8431629532546964, recall: 0.8354978354978355, f1: 0.8393128941074146, auc: 0.7540219949094751, fbeta: 0.7499345850222896


[I 2025-03-25 19:40:56,062] Trial 98 finished with value: 0.759916921999092 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 474, 'max_depth': 847, 'min_gain_to_split': 0.7070314670645013, 'min_data_in_leaf': 35, 'lambda_l1': 20.22055469194055, 'lambda_l2': 86.25786768121748, 'num_boost_round': 1702}. Best is trial 98 with value: 0.759916921999092.


precision: 0.8421052631578947, recall: 0.8588744588744589, f1: 0.8504072010287184, auc: 0.7552466023147482, fbeta: 0.759916921999092


[I 2025-03-25 19:41:43,853] Trial 99 finished with value: 0.7585766046461618 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 473, 'max_depth': 832, 'min_gain_to_split': 0.7105544482882309, 'min_data_in_leaf': 33, 'lambda_l1': 21.35104088005627, 'lambda_l2': 87.80136039807391, 'num_boost_round': 1721}. Best is trial 98 with value: 0.759916921999092.


precision: 0.8430953398888413, recall: 0.8536796536796537, f1: 0.8483544848354485, auc: 0.7543670803575717, fbeta: 0.7585766046461618


In [6]:
study.best_params

{'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 474,
 'max_depth': 847,
 'min_gain_to_split': 0.7070314670645013,
 'min_data_in_leaf': 35,
 'lambda_l1': 20.22055469194055,
 'lambda_l2': 86.25786768121748,
 'num_boost_round': 1702}

In [7]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [8]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
model.save_model("model/model_meta.txt")
prod_model.save_model("model/model_meta_prod.txt")